# Module 4

### Query4.1 
Import the csv file of the stock which contained the Bollinger columns as well.

* Create a new column 'Call' , whose entries are - 
  * 'Buy' if the stock price is below the lower Bollinger band 
  * 'Hold Buy/ Liquidate Short' if the stock price is between the lower and middle Bollinger band 
  * 'Hold Short/ Liquidate Buy' if the stock price is between the middle and upper Bollinger band 
  * 'Short' if the stock price is above the upper Bollinger band
  
* Now train a classification model with the 3 bollinger columns and the stock price as inputs and 'Calls' as output. Check the accuracy on a test set. (There are many classifier models to choose from, try each one out and compare the accuracy for each)

* Import another stock data and create the bollinger columns. Using the already defined model, predict the daily calls for this new stock.


In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

In [2]:
df = pd.read_csv('Small_Cap/FORTIS.csv')
df['Average'] = pd.read_csv("week2 v.csv")
df['+2 SD'] = pd.read_csv('week2 vu.csv')
df['-2 SD'] = pd.read_csv('week2 vl.csv')
df = df.dropna()
df['Date'] = df['Date'].astype('datetime64')
df.set_index('Date',inplace =True)
df.head()

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Average,+2 SD,-2 SD
Date,,,,,,,,,,,,,,,,,
2017-06-01,FORTIS,EQ,195.15,195.90,200.25,195.15,199.80,198.35,197.35,837394,1.652597e+08,6033,248194,29.64,199.871429,209.101539,190.641318
2017-06-02,FORTIS,EQ,198.35,200.05,201.60,191.75,193.20,192.95,195.33,2967855,5.797230e+08,15457,1540619,51.91,198.992857,208.358166,189.627548
2017-06-05,FORTIS,EQ,192.95,194.00,196.90,193.60,194.75,194.90,195.37,1688838,3.299437e+08,12741,654094,38.73,198.064286,206.110253,190.018319
2017-06-06,FORTIS,EQ,194.90,195.40,195.45,189.10,190.40,191.05,192.04,3708378,7.121618e+08,17057,2344287,63.22,197.003571,204.496077,189.511066
2017-06-07,FORTIS,EQ,191.05,191.10,198.70,191.10,194.50,194.05,195.63,3368728,6.590180e+08,21279,1095546,32.52,196.803571,204.461394,189.145748


In [12]:
df['Call'] = np.nan
df['Call'] = df['Call'].astype('str')

for i in range(len(df)):
    if df['Close Price'][i] < df['-2 SD'][i]:
        df['Call'][i] = 'Buy'
    elif df['Close Price'][i] > df['-2 SD'][i] and df['Close Price'][i] < df['Average'][i] :
        df['Call'][i] = 'Hold Buy/ Liquidate Short'
    elif df['Close Price'][i] < df['+2 SD'][i] and df['Close Price'][i] > df['Average'][i]:
        df['Call'][i] = 'Hold Short/ Liquidate Buy'
    elif df['Close Price'][i] > df['+2 SD'][i] :
        df['Call'][i] = 'Short'
    
df.head()

,Symbol,Series,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,Average,+2 SD,-2 SD,Call
Date,,,,,,,,,,,,,,,,,,
2017-06-01,FORTIS,EQ,195.15,195.90,200.25,195.15,199.80,198.35,197.35,837394,1.652597e+08,6033,248194,29.64,199.871429,209.101539,190.641318,Hold Buy/ Liquidate Short
2017-06-02,FORTIS,EQ,198.35,200.05,201.60,191.75,193.20,192.95,195.33,2967855,5.797230e+08,15457,1540619,51.91,198.992857,208.358166,189.627548,Hold Buy/ Liquidate Short
2017-06-05,FORTIS,EQ,192.95,194.00,196.90,193.60,194.75,194.90,195.37,1688838,3.299437e+08,12741,654094,38.73,198.064286,206.110253,190.018319,Hold Buy/ Liquidate Short
2017-06-06,FORTIS,EQ,194.90,195.40,195.45,189.10,190.40,191.05,192.04,3708378,7.121618e+08,17057,2344287,63.22,197.003571,204.496077,189.511066,Hold Buy/ Liquidate Short
2017-06-07,FORTIS,EQ,191.05,191.10,198.70,191.10,194.50,194.05,195.63,3368728,6.590180e+08,21279,1095546,32.52,196.803571,204.461394,189.145748,Hold Buy/ Liquidate Short


In [81]:
def accuracy(cm):
    num = cm.trace()
    deno = cm.sum()
    return num/deno

In [89]:
X = df.iloc[:,[7,-4,-3,-2]].values
y = df.iloc[:,-1:].values

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2,random_state = 5)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

from sklearn.metrics import confusion_matrix

In [93]:
#Logistic Rergression

from sklearn.linear_model import LogisticRegression
classifer_logi = LogisticRegression(random_state = 5)
classifer_logi.fit(X_train,y_train)

y_pred_logi = classifer_logi.predict(X_test)

cm_logi = confusion_matrix(y_test,y_pred_logi)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_logi)))

The accuracy of logistic model classifier is :0.9587628865979382


In [94]:
# KNN

from sklearn.neighbors import KNeighborsClassifier
classifer_KNN = KNeighborsClassifier(n_neighbors = 3)
classifer_KNN.fit(X_train,y_train)

y_pred_KNN = classifer_KNN.predict(X_test)

cm_KNN = confusion_matrix(y_test,y_pred_KNN)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_KNN)))

The accuracy of logistic model classifier is :0.9278350515463918


In [95]:
# Kernel SVM
from sklearn.svm import SVC
classifer_kSVC = SVC(kernel='rbf', random_state = 5)
classifer_kSVC.fit(X_train,y_train)

y_pred_kSVC = classifer_kSVC.predict(X_test)

cm_kSVC = confusion_matrix(y_test,y_pred_kSVC)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_kSVC)))

The accuracy of logistic model classifier is :0.9484536082474226


In [96]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifer_NB = GaussianNB()
classifer_NB.fit(X_train,y_train)

y_pred_NB = classifer_NB.predict(X_test)

cm_NB = confusion_matrix(y_test,y_pred_NB)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_NB)))

The accuracy of logistic model classifier is :0.4948453608247423


In [97]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifer_DT = DecisionTreeClassifier()
classifer_DT.fit(X_train,y_train)

y_pred_DT = classifer_DT.predict(X_test)

cm_DT = confusion_matrix(y_test,y_pred_DT)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_DT)))

The accuracy of logistic model classifier is :0.8762886597938144


In [92]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
classifer_RF = RandomForestClassifier()
classifer_RF.fit(X_train,y_train)

y_pred_RF = classifer_RF.predict(X_test)

cm_RF = confusion_matrix(y_test,y_pred_RF)
print("The accuracy of logistic model classifier is :" + str(accuracy(cm_RF)))

The accuracy of logistic model classifier is :0.9484536082474226


Now, we'll again utilize classification to make a trade call, and measure the efficiency of our trading algorithm over the past two years. For this assignment , we will use RandomForest classifier.
* Import the stock data file of your choice
  Define 4 new columns , whose values are: 
  % change between Open and Close price for the day 
  % change between Low and High price for the day 
  5 day rolling mean of the day to day % change in Close Price 
  5 day rolling std of the day to day % change in Close Price
* Create a new column 'Action' whose values are: 
  1 if next day's price(Close) is greater than present day's. 
  (-1) if next day's price(Close) is less than present day's. 
  i.e. Action [ i ] = 1 if Close[ i+1 ] > Close[ i ] 
  i.e. Action [ i ] = (-1) if Close[ i+1 ] < Close[ i ]
* Construct a classification model with the 4 new inputs and 'Action' as target
* Check the accuracy of this model , also , plot the net cumulative returns (in %) if we were to follow this algorithmic model

In [30]:
len(y_test)

97

In [34]:
print(cm.shape)

(4, 4)


In [78]:
X

array([[198.35      , 199.87142857, 209.10153935, 190.64131779],
       [192.95      , 198.99285714, 208.35816605, 189.62754823],
       [194.9       , 198.06428571, 206.11025256, 190.01831887],
       ...,
       [133.6       , 137.86428571, 142.78901586, 132.93955557],
       [133.        , 137.20357143, 142.68044087, 131.72670199],
       [131.9       , 136.43928571, 142.89376282, 129.98480861]])